# 🏃‍♂️ Garmin Shoe Performance Audit: Causal Inference in Personal Athletics

**Objective:** To determine the "true" performance impact of my running shoe rotation by controlling for physiological and environmental confounders.

### The Problem: "The Strava Bias"
Most running apps simply average your pace per shoe. This is flawed because:
1.  **Selection Bias:** I wear fast shoes on race day and slow shoes on recovery days.
2.  **Terrain Bias:** I wear stable shoes on hilly routes.
3.  **Fatigue:** A "slow" shoe might just be a victim of tired legs.

### The Solution: Dual-Model OLS Regression
To isolate the shoe's effect, we build a rigorous feature set (controlling for fitness, fatigue, and elevation) and run two competing models:
* **Model A (Mechanical Gain):** Controls for **Stride Length & Cadence**. This asks: *"If I run exactly the same way, does the shoe make me faster?"* (Pure efficiency).
* **Model B (Net Performance):** Removes biomechanical controls. This asks: *"Does the shoe encourage me to change my stride (e.g., higher turnover) to go faster?"* (Total system performance).

In [1]:
import json
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

# Load main activities file
file_path = r"C:\Users\kamro\dev\garmin-analysis\data\garmin data 123125\DI_CONNECT\DI-Connect-Fitness\kamrondaugherty@protonmail.com_0_summarizedActivities.json"
try:
    with open(file_path, 'r') as f:
        data = json.load(f)
    print("✅ Successfully loaded activities JSON.")
except Exception as e:
    print(f"Error loading activities: {e}")

# Load gear file
file_path2 = r"C:\Users\kamro\dev\garmin-analysis\data\garmin data 123125\DI_CONNECT\DI-Connect-Fitness\kamrondaugherty@protonmail.com_gear.json"
try:
    with open(file_path2, 'r') as g:
        data2 = json.load(g)
    print("✅ Successfully loaded gear JSON.")
except Exception as e:
    print(f"Error loading gear: {e}")

✅ Successfully loaded activities JSON.
✅ Successfully loaded gear JSON.


In [2]:
# Create main activities DataFrame (df)
activities_list = data[0]["summarizedActivitiesExport"] 
df = pd.DataFrame(activities_list) 

# Create gear details DataFrame (df2)
gear_list = data2[0]['gearDTOS'] 
df2 = pd.json_normalize(gear_list) 

# Fix and create gear-activity link DataFrame (df1_fixed)
gear_activity_dict = data2[0]['gearActivityDTOs'] 
list_of_lists = gear_activity_dict.values() 
all_gear_activities = [item for sublist in list_of_lists for item in sublist] 
df1_fixed = pd.json_normalize(all_gear_activities) 

# Merge gear data (df1_fixed + df2)
df_gear_details = pd.merge(df1_fixed, df2, on='gearPk', how='left') 

# Merge gear with main activities (df + df_gear_details)
gear_to_add = df_gear_details[['activityId', 'displayName']] 
df_final = pd.merge(df, gear_to_add, on='activityId', how='left') 

print("✅ All data loaded and merged into 'df_final'.")

# Verification step
print("\nVerifying the merge by checking shoe usage counts:")
print(df_final['displayName'].value_counts(dropna=False)) 

✅ All data loaded and merged into 'df_final'.

Verifying the merge by checking shoe usage counts:
displayName
NaN             286
Guide 15s       109
Adrenaline       67
Tempus           60
Launch 9s        52
Speed 3          24
Dualsport 2       7
gts launch 8      4
Name: count, dtype: int64


In [3]:
# Create a new DataFrame containing only 'running' activities
df_runs = df_final[df_final['activityType'] == 'running'].copy() 

print(f"\nTotal activities: {len(df_final)}") 
print(f"Total runs: {len(df_runs)}") 


Total activities: 609
Total runs: 363


## 🧪 Feature Engineering: Removing Confounders

To compare shoes fairly, we must mathematically flatten the playing field. We engineer four specific features to handle the biases identified above:

1.  **Aerobic Power Factor (PCA):**
    * *The Problem:* `Efficiency` (Watts/HR) and `VO2 Max` are highly correlated (multicollinearity).
    * *The Fix:* We use Principal Component Analysis (PCA) to collapse them into a single "Engine Score" that represents my physiological capacity on that day.

2.  **Training Stress Balance (TSB):**
    * *The Problem:* "Fitness" and "Fatigue" are opposing forces.
    * *The Fix:* We split mileage into two variables:
        * `fitness_42_day_km`: Long-term chronic load (makes me faster).
        * `fatigue_7_day_km`: Short-term acute load (makes me slower).

3.  **Relative Climb:**
    * *The Fix:* `Elevation Gain / Distance`. This normalizes hill difficulty so a shoe isn't penalized for being worn on a climbing day.

4.  **Data Leakage Prevention:**
    * *The Fix:* All rolling metrics (fitness/fatigue) are `.shift(1)`. We only care about my physical state *before* the run started, not after.

In [4]:
# --- ATOMIC FEATURE ENGINEERING v2.0 ---

# 1. Start fresh and sort
df_features = df_runs.copy().sort_values(by='beginTimestamp')
df_features['run_date'] = pd.to_datetime(df_features['beginTimestamp'], unit='ms')
df_features = df_features.set_index('run_date')

# 2. Distance and Unit Correction
df_features['total_distance_km'] = df_features['distance'] / 100000.0

# 3. FIX DATA LEAKAGE: Shift(1) rolling metrics
# This ensures fatigue/fitness is what you felt BEFORE the run started
df_features['fatigue_7_day_km'] = (
    df_features['total_distance_km'].rolling('7D').sum().shift(1).fillna(0)
)
df_features['fitness_42_day_km'] = (
    df_features['total_distance_km'].rolling('42D').sum().shift(1).fillna(0)
)

# 4. Normalized Elevation (Physical Therapist recommendation)
# Avoids bias where long runs look 'harder' just because of total cumulative gain
df_features['relative_climb'] = df_features['elevationGain'] / df_features['total_distance_km']

# 5. Shoe Mileage & Age Interactions (Shoe Engineer recommendation)
df_features['shoe_mileage_at_run'] = (
    df_features.groupby('displayName')['total_distance_km'].cumsum() - df_features['total_distance_km']
)
df_features['shoe_mileage_sq'] = df_features['shoe_mileage_at_run']**2

# 6. Efficiency and Categorical Cleanup
df_features['efficiency'] = df_features.apply(
    lambda row: row['avgPower'] / row['avgHr'] if row['avgHr'] > 0 else 0, axis=1
)

# Clean column names for statsmodels formula compatibility
df_features.columns = df_features.columns.str.replace(' ', '_', regex=False).str.replace('.', '_', regex=False)

# 7. Run Type Dummy logic
def categorize_run(name_str):
    if not isinstance(name_str, str): return 'Other'
    name_str_lower = name_str.lower()
    if 'base' in name_str_lower: return 'Base'
    if 'sprint' in name_str_lower: return 'Sprint'
    return 'Other'

df_features['run_type'] = df_features['name'].apply(categorize_run)

print("✅ Feature engineering v2.0 complete: Leakage removed, elevation normalized.")

✅ Feature engineering v2.0 complete: Leakage removed, elevation normalized.


## ⚔️ The Dual-Model Audit

We now run two regressions to distinguish between **mechanical assistance** and **behavioral adaptation**.

### Model A: The "Wind Tunnel" Test (Mechanical Gain)
$$Speed \sim Shoe + Power + \text{Fitness} + \text{Fatigue} + \mathbf{Stride} + \mathbf{Cadence}$$
* **Control Level:** High.
* **Interpretation:** If a shoe has a positive coefficient here, it provides "free speed" via foam geometry or energy return, independent of my mechanics.

### Model B: The "Real World" Test (Net Performance)
$$Speed \sim Shoe + Power + \text{Fitness} + \text{Fatigue}$$
* **Control Level:** Moderate (Biometrics removed).
* **Interpretation:** This allows the shoe to take credit for changing my gait. If the **Saucony Endorphin Speed 3** is insignificant in Model A but significant here, it means the shoe makes me faster by *forcing* a faster turnover or longer stride.

In [5]:
import statsmodels.formula.api as smf
from sklearn.decomposition import PCA

# 1. PREP & CLEANING (Maintaining all performance variables)
metrics_df = df[['activityId', 'vO2MaxValue', 'avgRunCadence', 'avgStrideLength']].copy()
metrics_df.columns = ['activityId', 'vo2_max', 'avg_cadence', 'avg_stride_length']

df_audit = pd.merge(df_features, metrics_df, on='activityId', how='left')
df_audit['displayName'] = df_audit['displayName'].str.replace(' ', '_', regex=False)
df_audit['displayName'] = df_audit['displayName'].replace('Launch_9s', 'GTS_Launch_8')

# Filtering for your primary rotation
primary_shoes = ['Guide_15s', 'Speed_3', 'Tempus', 'Adrenaline_', 'GTS_Launch_8']
df_clean = df_audit[df_audit['displayName'].isin(primary_shoes)].dropna(
    subset=['efficiency', 'vo2_max', 'avg_stride_length', 'avg_cadence', 'relative_climb']
).copy()

# 2. PCA: Aerobic Power (Combines Efficiency + VO2 Max to reduce collinearity)
scaler = StandardScaler()
scaled_aerobic = scaler.fit_transform(df_clean[['efficiency', 'vo2_max']])
df_clean['aerobic_power_factor'] = PCA(n_components=1).fit_transform(scaled_aerobic)

# --- MODEL A: MECHANICAL EFFICIENCY (Controlled for Stride/Cadence) ---
# This shows: "If I run with the exact same stride, which shoe is faster?"
formula_A = """
avgSpeed ~ C(displayName, Treatment(reference='Guide_15s')) + 
           aerobic_power_factor + avg_stride_length + avg_cadence + 
           fitness_42_day_km + fatigue_7_day_km + relative_climb
"""

# --- MODEL B: TOTAL SHOE PERFORMANCE (Uncontrolled for Stride/Cadence) ---
# This shows: "Which shoe makes me faster overall, potentially by improving my stride?"
formula_B = """
avgSpeed ~ C(displayName, Treatment(reference='Guide_15s')) + 
           aerobic_power_factor + 
           fitness_42_day_km + fatigue_7_day_km + relative_climb
"""

model_A = smf.ols(formula=formula_A, data=df_clean).fit()
model_B = smf.ols(formula=formula_B, data=df_clean).fit()

print("--- OUTPUT 1: MECHANICAL GAIN (Stride & Cadence Controlled) ---")
print(model_A.summary())

print("\n\n--- OUTPUT 2: NET SHOE GAIN (Stride & Cadence Excluded) ---")
print(model_B.summary())

--- OUTPUT 1: MECHANICAL GAIN (Stride & Cadence Controlled) ---
                            OLS Regression Results                            
Dep. Variable:               avgSpeed   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                     3904.
Date:                Thu, 01 Jan 2026   Prob (F-statistic):          3.27e-275
Time:                        21:02:11   Log-Likelihood:                 1323.8
No. Observations:                 269   AIC:                            -2626.
Df Residuals:                     258   BIC:                            -2586.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
-------------